# Chapter 1 - Introduction / Business Problem 

## Housing shortage in the Netherlands
Young people in the Netherlands are strugling to buy their first home. There is a limited supply available and investers buy a lot of it to convert to cheap rentals. Thus leaving the first time buyers with little chance of actually obtaining there first home. It's really frustrating for this group and the government is trying to help them out, with little succes so far. 

We see a lot of these first time buyers looking for a house in a certain area and if they are unable to actually buy something the will broaden there search area. The analyses in this report will help these first time buyers, and everyone else looking for a new home to broaden there horizon. Find different areas where housing prices may be lower, and the markter less overheated while still having more or less the same services available. This will dramatically increase there chance of finding a suitable home and making a good personal investment. 

### Densely populated country
The Netherlands is a densely populated country with over 412.49 people per sqaure kilometer (p/km2). As a comparison the USA has a population density of 33.67 p/km2. Earning a place in the top 20 of most densely populated countries. 

https://www.populationpyramid.net/population-density/2020/

### Growing population
The population in the Netherlands is still growing. Allthough the rate at which it is increasing is getting slower. Every year there is a need for more houses, conflicting with other interests like nature, agriculture, office space, airfields, windmills, etc. 

https://www.worldometers.info/world-population/netherlands-population/

### Decreasing household size 
There is a clearly visable trend in the size of the average household size (number of people in a household) in the last decades. The household size is shrinking steadily from 1970 until present day. More and more single person households or couples without children are buying or renting houses. The number of households keeps on increasing even if the total population would stay the same. The increase in households means we need more homes for the same number of people. 

https://themasites.pbl.nl/balansvandeleefomgeving/jaargang-2014/wonen-en-vastgoed/passendheid-conclusies/passendheid-huishoudens

### Is the whole of the Netherlands overpopulated? 
On general the population of the Netherlands is increasing but there are a lot of regional exeptions. There are areas of the Netherlands where the population is shrinking. These places would welcome the influx of new people! Because who wants to live in a ghost town? In the period 2015-2020 the area's near the borders are the primary areas with a decrease in population density.

https://www.cbs.nl/nl-nl/dossier/dossier-verstedelijking/hoofdcategorieen/waar-groeit-of-krimpt-de-bevolking-

## Research questions: Where the hell can I buy a house in the Netherlands? 

#### Why are some area's more atractive then others? 
#### What are the differences and similarieties between the municipallities (or clusters)?
#### Is there a correlation between the housing price and population growth?
#### Can we predict population growth based on location data?
#### How accurate can we model population growth/shirinkage based on location data? 
#### What is causing population growth of shrinkage?  
#### Can we reverse depopulation by stimulating certain venues or services?


# Chapter 2- The Data
In this research I will be combining data from two different sources to be able to answer the research questions. 

## Part one -  Central bureau of statistics (CBS) 
From the CBS we will get data about the municipalities, including population growth and density, avarage house prices, coordinates, etc. This will be the starting point for us in building a data frame to answer our research questions. This data is publically available on https://www.cbs.nl/. It is all structered tabular data in CSV file. To be able to use the data I will combine differend data sets and clean the data to be able to properly work with it. 

## Part two - Foursquare 
Foursquare will provide us with additional data about available venues in these municipalities. Using the Foursquare API I will gather information about (popular) venues in the municipalities. Based on the most popular venues the municipalities will be clustered together using a KNN algorithm. We will add the location data and clusters to the municipality dataframe. 

# Cleaning the data

## Part one - CBS 

### Municipallity dataframe from CBS data
To start with will load, clean en merge several CSV files into one dataframe. This will be the backbone for our further analyses. 

In [3]:
## Importing libraries

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files
from pandas import json_normalize # tranform JSON file into a pandas dataframe
import requests # to communicate with API

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library folium
#!conda install -c conda-forge Folium=0.5.0 --yes 
import folium 

print('libraries imported succesfully')

libraries imported succesfully


In [5]:
# The code was removed by Watson Studio for sharing.

,Naam,Type,Provincie afkorting,Bezoekadres straat,Bezoekadres huisnummer,Bezoekadres toevoeging,Bezoekadres postcode,Bezoekadres plaats,Bezoekadres latitude,Bezoekadres longitude,Oppervlakte (km2),Aantal inwoners,Inwoners per km2
0,Aa en Hunze,Gemeente,DR,Spiekersteeg,1,NaN,9461 BH,GIETEN,53.005,6.760,279,25390,90
1,Aalsmeer,Gemeente,NH,Raadhuisplein,1,NaN,1431 EH,AALSMEER,52.267,4.750,34,31499,928
2,Aalten,Gemeente,GD,Hofstraat,8,NaN,7121 DM,AALTEN,51.926,6.583,97,26962,277
3,Achtkarspelen,Gemeente,FR,Stationsstraat,18,NaN,9285 NH,BUITENPOST,53.255,6.143,104,27935,268
4,Alblasserdam,Gemeente,ZH,Cortgene,2,NaN,2951 ED,ALBLASSERDAM,51.862,4.658,10,20014,2001


In [6]:
## Cleaning the dataframe
df_mun = df_CBS[['Naam', 'Bezoekadres latitude', 'Bezoekadres longitude', "Oppervlakte (km2)", 
        'Aantal inwoners', "Inwoners per km2"]] 

df_mun = df_mun.rename(columns={"Naam":"municipallity", "Bezoekadres latitude":"latitude", 
                       'Bezoekadres longitude':'longitude', "Oppervlakte (km2)": "surface area (km2)", 
                      'Aantal inwoners':"population", "Inwoners per km2" : "population density per km2"})

df_mun.info() # first check on the dataframe we can see some municipallities have no location data
df_mun.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   municipallity               352 non-null    object 
 1   latitude                    344 non-null    float64
 2   longitude                   344 non-null    float64
 3   surface area (km2)          352 non-null    int64  
 4   population                  352 non-null    int64  
 5   population density per km2  352 non-null    int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 16.6+ KB


,municipallity,latitude,longitude,surface area (km2),population,population density per km2
0,Aa en Hunze,53.005,6.760,279,25390,90
1,Aalsmeer,52.267,4.750,34,31499,928
2,Aalten,51.926,6.583,97,26962,277
3,Achtkarspelen,53.255,6.143,104,27935,268
4,Alblasserdam,51.862,4.658,10,20014,2001


In [7]:
# drop municipallities without coordinates 
df_mun = df_mun.dropna() 
df_mun.describe() # Shows there is at least one municipallity with a surface area of 0 km 

,latitude,longitude,surface area (km2),population,population density per km2
count,344.000000,344.000000,344.000000,344.000000,3.440000e+02
mean,52.048712,5.394965,103.970930,48316.843023,4.168147e+04
std,0.560028,0.766609,107.316943,72488.068116,5.451017e+05
min,50.771000,3.487000,0.000000,932.000000,3.000000e+00
25%,51.647250,4.799500,33.000000,21252.750000,2.267500e+02
50%,52.026000,5.410000,73.000000,29821.500000,4.385000e+02
75%,52.379000,5.965750,124.500000,48346.500000,1.099500e+03
max,53.479000,7.071000,765.000000,854047.000000,8.458800e+06


In [8]:
df_mun.loc[df_mun["surface area (km2)"] == 0] 
## there are two municipallities with a surface area of 0 and a very large population density as a result
## in order to be able to proceed our analyses we will drop these rows 


,municipallity,latitude,longitude,surface area (km2),population,population density per km2
159,Krimpenerwaard,51.973,4.771,0,55644,5564400
204,Nissewaard,51.848,4.329,0,84588,8458800


In [9]:
df_mun.drop(index = [159, 204], inplace=True)

In [10]:
df_mun.reset_index(drop=True, inplace=True)
df_mun.describe()

,latitude,longitude,surface area (km2),population,population density per km2
count,342.000000,342.000000,342.000000,342.000000,342.000000
mean,52.049520,5.399906,104.578947,48189.362573,921.710526
std,0.561548,0.765923,107.334533,72672.598162,1803.437871
min,50.771000,3.487000,1.000000,932.000000,3.000000
25%,51.643750,4.801500,34.000000,21226.250000,226.250000
50%,52.030500,5.416500,73.000000,29754.000000,436.500000
75%,52.383000,5.967250,125.500000,47714.000000,1085.500000
max,53.479000,7.071000,765.000000,854047.000000,28637.000000


In [11]:
from math import sqrt #import the function to calculate the sqaure root

for ind, row in df_mun.iterrows():
    df_mun.loc[ind, 'approximate radius (m)'] = sqrt(row['surface area (km2)'] / 3.142) * 1000 ## We approximated the radius of the municipallity by acting as if it are all perfect circles. In this way we will more or less get the average radius.
df_mun['approximate radius (m)'] = df_mun['approximate radius (m)'].astype(int)
    
df_mun.head()

,municipallity,latitude,longitude,surface area (km2),population,population density per km2,approximate radius (m)
0,Aa en Hunze,53.005,6.760,279,25390,90,9423
1,Aalsmeer,52.267,4.750,34,31499,928,3289
2,Aalten,51.926,6.583,97,26962,277,5556
3,Achtkarspelen,53.255,6.143,104,27935,268,5753
4,Alblasserdam,51.862,4.658,10,20014,2001,1784


### importing average house prices
Also from the CBS but in a different dataframe

In [12]:

body = client_6d61fec0a3dd4c5a8549fe3300270ebc.get_object(Bucket='datasciencecapstoneprojectbattleo-donotdelete-pr-wcot4qbo8rnhhg',Key='gemiddelde huizenprijzen.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_house_price = pd.read_csv(body, sep = ';')
df_house_price.head()


,Gemeente,Gemiddelde verkoopprijs ( duizend euro),house price (euro)
0,Appingedam,"194,8",194800
1,Delfzijl,"155,1",155100
2,Groningen,"258,9",258900
3,Loppersum,"192,4",192400
4,Almere,"294,3",294300


In [14]:
df_house_price = df_house_price.rename(columns={"Gemeente":"municipallity", "Gemiddelde verkoopprijs ( duizend euro)":
                                                "average house price (1000 euro)"})
df_house_price = df_house_price[['municipallity', "house price (euro)"]]
df_house_price.head()

,municipallity,house price (euro)
0,Appingedam,194800
1,Delfzijl,155100
2,Groningen,258900
3,Loppersum,192400
4,Almere,294300


In [15]:
df_cbs = pd.merge(df_mun, df_house_price)
df_cbs.describe()

,latitude,longitude,surface area (km2),population,population density per km2,approximate radius (m),house price (euro)
count,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000
mean,52.049964,5.396339,105.153153,46191.873874,906.204204,5207.312312,310388.588589
std,0.556063,0.768732,108.436912,67541.780748,1799.780005,2522.904249,85320.174747
min,50.771000,3.487000,1.000000,932.000000,3.000000,564.000000,161300.000000
25%,51.649000,4.801000,33.000000,21266.000000,226.000000,3240.000000,258000.000000
50%,52.033000,5.405000,73.000000,29753.000000,432.000000,4820.000000,293700.000000
75%,52.377000,5.968000,127.000000,47537.000000,1075.000000,6357.000000,340900.000000
max,53.479000,7.071000,765.000000,854047.000000,28637.000000,15603.000000,831600.000000


In [16]:
df_cbs.head(10)

,municipallity,latitude,longitude,surface area (km2),population,population density per km2,approximate radius (m),house price (euro)
0,Aa en Hunze,53.005,6.760,279,25390,90,9423,284800
1,Aalsmeer,52.267,4.750,34,31499,928,3289,409300
2,Aalten,51.926,6.583,97,26962,277,5556,226100
3,Achtkarspelen,53.255,6.143,104,27935,268,5753,216500
4,Alblasserdam,51.862,4.658,10,20014,2001,1784,268400
5,Albrandswaard,51.861,4.394,24,25218,1050,2763,361900
6,Alkmaar,52.635,4.746,31,108470,3474,3141,286900
7,Almelo,52.357,6.659,69,72629,1046,4686,210000
8,Almere,52.372,5.222,249,203990,819,8902,294300
9,Alphen aan den Rijn,52.129,4.660,58,109682,1901,4296,286500


### Population growth
Below we are going to add the population growth or shrinkage in the periode of 2015 - 2020 to our dataframe. This is the last step in completing the CBS data.

In [17]:

body = client_6d61fec0a3dd4c5a8549fe3300270ebc.get_object(Bucket='datasciencecapstoneprojectbattleo-donotdelete-pr-wcot4qbo8rnhhg',Key='population growth.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_pop_growth = pd.read_csv(body, sep = ';')


In [18]:
df_pop_growth = df_pop_growth.rename(columns={"Gemeente":"municipallity", "relatieve ontwikkeling 2015-2020 (%)":
                                                "pop"})
df_pop_growth ['population growth (%)'] = df_pop_growth['pop'].str.replace(',','.').astype(float)
df_pop_growth = df_pop_growth [["municipallity",'population growth (%)']]
df_pop_growth.head()

,municipallity,population growth (%)
0,Appingedam,-3.07
1,Delfzijl,-2.88
2,Groningen,2.44
3,Loppersum,-5.95
4,Almere,7.60


In [19]:
df_cbs = pd.merge(df_cbs, df_pop_growth)
df_cbs.head()

,municipallity,latitude,longitude,surface area (km2),population,population density per km2,approximate radius (m),house price (euro),population growth (%)
0,Aa en Hunze,53.005,6.760,279,25390,90,9423,284800,0.96
1,Aalsmeer,52.267,4.750,34,31499,928,3289,409300,2.52
2,Aalten,51.926,6.583,97,26962,277,5556,226100,0.81
3,Achtkarspelen,53.255,6.143,104,27935,268,5753,216500,-0.50
4,Alblasserdam,51.862,4.658,10,20014,2001,1784,268400,1.61


## CBS data description
Above you can see a part of the dataframe which we will be using in our furter analyses. It is a dataframe containing 9 columns of relevant data which we will discuss in detail below. In total we have this information on 333 municipallities in the Netherlands. The dataframe is orded alphabetically based on the municipallity name.

##### Column 1 - municipallity
This is the name of the municipallities used by the central bureau of statistics of the Netherlands. 

##### Column 2 & 3 - Latitude and Longitude
These are the central coordinates of the different municipallities. They will be used as input for obtaining the foursquare venue data as well as plotting different maps as data visualisations.

##### column 4 - Surface area (km2)
The different municipallities have a quite a big range in de size. Bigger municipallities might have more diversification and als more available veneus. On itself a interesting parameter for further analyses and in combination with column 5 it give the avarega population density. The surface area is noted in square kilometers.

##### column 5  - population
The total number of people living in this municipallity. More people often means more money and more services. But als combined with column 4 an input for population density.

##### column 6 - population density per km2
This is the average amount of people living in one square kilometer area in this municipallity. A higher population density will be found in urban settings then in rural ones. Als an indicator for how the amount of agricultural and nature compared to housing. Also a higher population density is often combined with more available services. 

##### column 7 - approximate radius
Due to the differt sizes of the municipallities we wanted to change the radius for each API call to foursqaure to get appropriate data for each municipallity. If this will not work I will use the average of the approximate radius as input. The approximate radius is calculated by acting as if all the municipallities are perfect circles and obtaining a radius by converting the surface area of the circle to its radius. In general this should give a good approximation of the average radius of the municipallity. 

##### column 8 - house price (euro)
This is the average houseprice per municipallity as registered by the CBS. 

##### column 9 - population growth (%)
On general the population of the Netherlans is growing. This column gives an indication of how that growth is distributed throughout the municipallities. The data is based on the periode of 2015 - 2020, calculating the difference in the population in the beginning and the end of this time periode. The negative numbers are areas with population shrinkage while the positives are growth. 

## Part Two - location data using Foursquare 
The CBS data discribe in the first part of the chapter will be used to obtain data from Foursquare. First we will visualise the spatial data using a folium map to get an idea of where we are located. In a following chapter we will be using the API to collect venue data for the different municipallities. 

Foursquare is an extensive geospatial database which provide a lot of spatial data to differt companies and services. The data is continualy updated and validated by millions of consumers. For this research I am interesseted in POI's (points of interest) in the different municipallities. With the extanded database of foursqaure I will be able to obtain an large amount of relavant real life data in no time. 

According to Foursquare; 
"If it tells you where, it’s probably built on Foursquare. We believe in the power of location. What people experience in the real world and the places they go are powerful reflections of who they are and what they care about. We help leading global companies tap into this intelligence to create better customer experiences and smarter business outcomes, all based on the world’s leading platform for understanding people, places, and the interactions between them." 

In a following chapter I will use the foursquare data bases to querry for relavant venue data. This will let me explore all the nearby venues and it's characteristics. For this research I'm interested in what kind of venues are available and what will distinguish one municiapallity from another. 

The location data from foursquare will also be the input for our KNN clustering to find similar municipallies and spatial distributions of the clusters. 


In [20]:
# finding the centre of our data points to zoom into on the map
latitude_netherlands = df_cbs['latitude'].mean()
longitude_netherlands = df_cbs['longitude'].mean()

print('coordinates of the netherlands are: ', latitude_netherlands, longitude_netherlands)

coordinates of the netherlands are:  52.04996396396396 5.396339339339339


In [21]:
# create map of Netherlans using latitude and longitude values of the municipallities to get a feel for the data

map_mun = folium.Map(location=[latitude_netherlands, longitude_netherlands], zoom_start=7)

# add markers to map
for lat, lng, mun in zip(df_cbs['latitude'], df_cbs['longitude'], df_cbs['municipallity']):
    label = '{}'.format(mun)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mun)  
    
map_mun

For every location in de Map above we will use the foursquare API to obtain data of the surrounding venues. From this venue data we will extract the venue categories, which will be stored in a dataframe.

### obtaining Foursquare location data

In [22]:
# The code was removed by Watson Studio for sharing.

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['municipallity', 
                  'municipallity Latitude', 
                  'municipallity Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
Venues = getNearbyVenues(df_cbs['municipallity'], df_cbs['latitude'], df_cbs['longitude'], radius)

Aa en Hunze
Aalsmeer
Aalten
Achtkarspelen
Alblasserdam
Albrandswaard
Alkmaar
Almelo
Almere
Alphen aan den Rijn
Alphen-Chaam
Altena
Ameland
Amersfoort
Amstelveen
Amsterdam
Apeldoorn
Arnhem
Assen
Asten
Baarle-Nassau
Baarn
Barendrecht
Barneveld
Beek
Beekdaelen
Beemster
Beesel
Berg en Dal
Bergeijk
Bergen op Zoom
Berkelland
Bernheze
Best
Beuningen
Beverwijk
Bladel
Blaricum
Bloemendaal
Bodegraven-Reeuwijk
Boekel
Borger-Odoorn
Borne
Borsele
Boxmeer
Boxtel
Breda
Brielle
Bronckhorst
Brummen
Brunssum
Bunnik
Bunschoten
Buren
Capelle aan den IJssel
Castricum
Coevorden
Cranendonck
Cuijk
Culemborg
Dalfsen
Dantumadiel
De Bilt
De Fryske Marren
De Ronde Venen
De Wolden
Delft
Den Helder
Deurne
Deventer
Diemen
Dinkelland
Doesburg
Doetinchem
Dongen
Dordrecht
Drechterland
Drimmelen
Dronten
Druten
Duiven
Echt-Susteren
Edam-Volendam
Ede
Eemnes
Eersel
Eijsden-Margraten
Eindhoven
Elburg
Emmen
Enkhuizen
Enschede
Epe
Ermelo
Etten-Leur
Geertruidenberg
Geldrop-Mierlo
Gemert-Bakel
Gennep
Gilze en Rijen
Goeree-Overf

In [ ]:
print(Venues.shape)
Venues.head(10)

### Foursquare data description
 
Above are the first 10 entries out of the location data dataframe. Besides the municipallity data this dataframe contains the name of the venue, its coordinates and the venue categorie. The venue catergory is the thing we are going to use in further analyses. The total dataframe consists of 7 columns with 5892 venues. 

##### column 1 Municipality 
This is the name of the municipallity in which the Venue is located. We can use this column to group the data and merge it into the CBS dataset. 

##### column 2 & 3 Municipality Latitude and Longitude
Coordinates of the municipallity

##### column 4 Venue
This is the name of the Venue as it is known in the Foursquare database. This is not a unique name as there can be different branches of the same shop, like 'Jumbo' supermarket that can be located in different towns or neighborhoods. 

##### column 5 & 6 Venue Latitude and Longitude
The coordinates of the actual venue that itself 

##### column 7 Venue Category 
In the foursquare database all Venues are put in a category. This will be the main location data that we are going to use in the rest of the analyses. As this describes what kind of services and venues are available in the differtent municipalities. 

